In [16]:
import sys
import pickle
# import cPickle
from datetime import datetime, timezone, timedelta
from enum import Enum
import numpy as np
import pandas as pd

from pathlib import Path
import matplotlib.colors as mcolors

# from PyQt5 import QtGui, QtWidgets
# from PyQt5.QtWidgets import QMessageBox, QToolTip, QStackedWidget, QHBoxLayout, QVBoxLayout, QSplitter, QFormLayout, QLabel, QFrame, QPushButton, QTableWidget, QTableWidgetItem, QScrollArea
# from PyQt5.QtWidgets import QApplication, QFileSystemModel, QTreeView, QWidget, QAction, qApp, QApplication, QTreeWidgetItem, QFileDialog 
# from PyQt5.QtGui import QPainter, QBrush, QPen, QColor, QFont, QIcon
# from PyQt5.QtCore import Qt, QPoint, QRect, QObject, QEvent, pyqtSignal, pyqtSlot, QSize, QDir, QThreadPool

# from phopyqttimelineplotter.GUI.UI.AbstractDatabaseAccessingWidgets import AbstractDatabaseAccessingQObject

from phopyqttimelineplotter.app.filesystem.VideoUtils import findVideoFiles, VideoParsedResults, FoundVideoFileResult, CachedFileSource
from phopyqttimelineplotter.app.filesystem.Workers.FileMetadataWorkers import FileMetadataWorker
from phopyqttimelineplotter.app.filesystem.Workers.VideoFilesystemWorkers import VideoFilesystemWorker

from phopyqttimelineplotter.GUI.Model.ModelViewContainer import ModelViewContainer
from phopyqttimelineplotter.app.filesystem.FilesystemOperations import OperationTypes, PendingFilesystemOperation
from phopyqttimelineplotter.app.filesystem.LabjackData.LabjackEventsLoader import LabjackEventsLoader, PhoServerFormatArgs

from phopyqttimelineplotter.app.filesystem.FilesystemRecordBase import FilesystemRecordBase, FilesystemLabjackEvent_Record
from phopyqttimelineplotter.GUI.Model.Events.PhoDurationEvent import PhoDurationEvent

from phopyqttimelineplotter.app.filesystem.LabjackData.LabjackFilesystemLoadingMixin import LabjackEventFile, LabjackFilesystemLoader

In [ ]:
## Event Loading:

## User-configurable settings:
# should_filter_for_invalid_events = True
should_filter_for_invalid_events = False
aFoundLabjackDataFile = Path("I:/EventData/BB01/out_file_s470017560_20190911-20190820_46Combined.csv")
videoStartDates = []
videoEndDates = []

active_cache = dict()
# LabjackEventFile: this serves as a container to hold the loaded events
outEventFileObj = LabjackEventFile(aFoundLabjackDataFile)

(dateTimes, labjackEventContainers, phoServerFormatArgs) = LabjackFilesystemLoader.loadLabjackEventsFile(aFoundLabjackDataFile, videoStartDates, videoEndDates, shouldLimitEventsToVideoDates=False, usePhoServerFormat=True, phoServerFormatIsStdOut=False, should_filter_for_invalid_events=should_filter_for_invalid_events)

print('Loading complete... setting loaded values')
# Cache the loaded values into the LabjackEventFile object.
# outEventFileObj.set_loaded_values(dateTimes, [], [], labjackEventContainers, phoServerFormatArgs)
outEventFileObj.set_loaded_values(dateTimes, [], [], labjackEventContainers, None)
print('done updating cache...')

if (not (aFoundLabjackDataFile in active_cache.keys())):
    # print('Creating new cache entry for {}...'.format(str(aFoundLabjackDataFile)))
    # Parent doesn't yet exist in cache
    active_cache[aFoundLabjackDataFile] = outEventFileObj
else:
    # Parent already exists
    print("WARNING: labjack file path {} already exists in the temporary cache. Updating its values...".format(str(aFoundLabjackDataFile)))
    active_cache[aFoundLabjackDataFile] = outEventFileObj
    pass

            

In [17]:
""" Import Dataframe from file:
"""
# dataframe_import_path = 'data/output/LabjackDataExport/output_dataframe_1-9-2020'
out_dataframe_export_parent_path = Path('data/output/LabjackDataExport/')
out_dataframe_export_path_basic = out_dataframe_export_parent_path.joinpath('output_dataframe_1-9-2020_basic_store.h5') # Used for basic objects
out_dataframe_export_path_pandas = out_dataframe_export_parent_path.joinpath('output_dataframe_1-9-2020_pandas_store.h5') # Used for pandas Dataframe and Series objects
        
# # Basic
# active_path = out_dataframe_export_path_basic
# print('Reading data from file {}...'.format(str(active_path)))
# active_store = pd.HDFStore(active_path)
# print(active_store)
# # in_df = active_store['variables_dataframe']
# active_store.close()
# print('done reading {}.'.format(str(active_path)))

# Pandas:
active_path = out_dataframe_export_path_pandas
print('Reading data from file {}...'.format(str(active_path)))
active_store = pd.HDFStore(active_path)
# print(active_store)
in_df = active_store['variables_dataframe']
in_series = active_store['variables_series_of_dataframes']
in_records_df = active_store['records_dataframe']

active_store.close()
print('done reading {}.'.format(str(active_path)))


# in_df
# for (aVariableIndex, aVariableData) in enumerate(variableData):
#     aVariableData['']
        
#         print('Converting variableData to Pandas Dataframe...')
#         out_df = get_variables_as_dataframe(variableData, active_labjack_variable_names)
#         # out_df.to_json(orient='split')

Reading data from file data\output\LabjackDataExport\output_dataframe_1-9-2020_pandas_store.h5...
done reading data\output\LabjackDataExport\output_dataframe_1-9-2020_pandas_store.h5.


In [ ]:
in_records_df

In [ ]:
in_records_df.columns

In [ ]:
in_records_df.extended_info_dict[0]

In [ ]:
in_records_df[['start_date', 'end_date', 'variable_name']]

In [ ]:
## https://stackoverflow.com/questions/38231591/splitting-dictionary-list-inside-a-pandas-column-into-separate-columns

# in_records_df.extended_info_dict.apply(pd.Series)[['event_type', 'dispense_type', 'port']]

In [ ]:
# in_records_df.drop(['b'], axis=1)

In [18]:
## Choose just the columns we care about. See https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe
# https://stackoverflow.com/questions/38231591/splitting-dictionary-list-inside-a-pandas-column-into-separate-columns
# https://stackoverflow.com/questions/34997174/how-to-convert-list-of-model-objects-to-pandas-dataframe/41762270

# pd.concat([in_records_df.drop(['b'], axis=1), in_records_df['b'].apply(pd.Series)], axis=1)
# final_out_df = pd.concat([in_records_df[['start_date', 'end_date', 'variable_name']], in_records_df.extended_info_dict.apply(pd.Series)[['event_type', 'dispense_type', 'port']]], axis=1)
final_out_df = pd.concat([in_records_df[['start_date', 'variable_name']], in_records_df.extended_info_dict.apply(pd.Series)[['event_type', 'port']]], axis=1)

In [ ]:
final_out_df

In [19]:
# out_dataframe_export_path_final_csv = out_dataframe_export_parent_path.joinpath('output_final_dataframe_1-9-2020.csv') # Used for basic objects
out_dataframe_export_path_final_csv = out_dataframe_export_parent_path.joinpath('output_final_dataframe_1-9-2020-Filtered.csv') # Used for basic objects

final_out_df.to_csv(out_dataframe_export_path_final_csv)

In [ ]:
in_series

In [ ]:
in_series.shape

In [ ]:
in_series.index

In [ ]:
# str(in_series.Water1_BeamBreak.variableSpecificRecords)
# get_extended_data()
in_series.Water1_BeamBreak.variableSpecificRecords[0].__dict__

In [ ]:
in_series.Water1_BeamBreak.timestamps[0]

In [ ]:
in_series.Water1_BeamBreak.values[0]

In [ ]:
for (recordIndex, aRecord) in in_series.Water1_BeamBreak.variableSpecificRecords:
    print(aRecord)

In [ ]:
pd.DataFrame(in_series, )

In [ ]:
pd.concat(in_series)

In [ ]:
pd.concat(in_series, keys=['s1', 's2'],  names=['Series name', 'Row ID'])

In [ ]:
in_df = in_df.T

In [ ]:
in_df.shape

In [ ]:
in_df.index

In [ ]:
in_df.columns

In [ ]:
in_df.dtypes

In [ ]:
in_df.variableSpecificRecords

In [ ]:
# in_df.axes
in_df.head(20)